# HW: X-ray images classification
--------------------------------------

Before you begin, open Mobaxterm and connect to triton with the user and password you were give with. Activate the environment `2ndPaper` and then type the command `pip install scikit-image`.

In this assignment you will be dealing with classification of 32X32 X-ray images of the chest. The image can be classified into one of four options: lungs (l), clavicles (c), and heart (h) and background (b). Even though those labels are dependent, we will treat this task as multiclass and not as multilabel. The dataset for this assignment is located on a shared folder on triton (`/MLdata/MLcourse/X_ray/'`).

In [1]:
import os
import numpy as np
from tensorflow.keras.layers import Dense, MaxPool2D, Conv2D, Dropout
from tensorflow.keras.layers import Flatten, InputLayer
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import *

from tensorflow.keras.initializers import Constant
from tensorflow.keras.datasets import fashion_mnist
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from skimage.io import imread

from skimage.transform import rescale, resize, downscale_local_mean
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options =
                         tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)
# device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

In [3]:
def preprocess(datapath):
    # This part reads the images
    classes = ['b','c','l','h']
    imagelist = [fn for fn in os.listdir(datapath)]
    N = len(imagelist)
    num_classes = len(classes)
    images = np.zeros((N, 32, 32, 1))
    Y = np.zeros((N,num_classes))
    ii=0
    for fn in imagelist:

        src = imread(os.path.join(datapath, fn),1)
        img = resize(src,(32,32),order = 3)
        
        images[ii,:,:,0] = img
        cc = -1
        for cl in range(len(classes)):
            if fn[-5] == classes[cl]:
                cc = cl
        Y[ii,cc]=1
        ii += 1

    BaseImages = images
    BaseY = Y
    return BaseImages, BaseY

In [4]:
def preprocess_train_and_val(datapath):
    # This part reads the images
    classes = ['b','c','l','h']
    imagelist = [fn for fn in os.listdir(datapath)]
    N = len(imagelist)
    num_classes = len(classes)
    images = np.zeros((N, 32, 32, 1))
    Y = np.zeros((N,num_classes))
    ii=0
    for fn in imagelist:

        images[ii,:,:,0] = imread(os.path.join(datapath, fn),1)
        cc = -1
        for cl in range(len(classes)):
            if fn[-5] == classes[cl]:
                cc = cl
        Y[ii,cc]=1
        ii += 1

    return images, Y

In [5]:
#Loading the data for training and validation:
src_data = '/MLdata/MLcourse/X_ray/'
train_path = src_data + 'train'
val_path = src_data + 'validation'
test_path = src_data + 'test'
BaseX_train , BaseY_train = preprocess_train_and_val(train_path)
BaseX_val , BaseY_val = preprocess_train_and_val(val_path)
X_test, Y_test = preprocess(test_path)

In [6]:
keras.backend.clear_session()

### PART 1: Fully connected layers 
--------------------------------------

---
<span style="color:red">***Task 1:***</span> *NN with fully connected layers. 

Elaborate a NN with 2 hidden fully connected layers with 300, 150 neurons and 4 neurons for classification. Use ReLU activation functions for the hidden layers and He_normal for initialization. Don't forget to flatten your image before feedforward to the first dense layer. Name the model `model_relu`.*

---

In [7]:
#--------------------------Impelment your code here:-------------------------------------
from tensorflow.keras.initializers import he_normal

model_relu = Sequential(name="model_relu")
model_relu.add(Flatten(input_shape=(32, 32, 1)))
model_relu.add(Dense(300, activation='relu', kernel_initializer=tf.keras.initializers.he_normal()))
model_relu.add(Dense(150))
model_relu.add(Activation('relu'))
model_relu.add(Dense(4))
model_relu.add(Activation('softmax'))
#----------------------------------------------------------------------------------------

In [8]:
model_relu.summary()

Model: "model_relu"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 300)               307500    
_________________________________________________________________
dense_1 (Dense)              (None, 150)               45150     
_________________________________________________________________
activation (Activation)      (None, 150)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 604       
_________________________________________________________________
activation_1 (Activation)    (None, 4)                 0         
Total params: 353,254
Trainable params: 353,254
Non-trainable params: 0
__________________________________________________

In [9]:
#Inputs: 
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 0
batch_size = 64
epochs = 25

#Define your optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)


Compile the model with the optimizer above, accuracy metric and adequate loss for multiclass task. Train your model on the training set and evaluate the model on the testing set. Print the accuracy and loss over the testing set.

In [10]:
#--------------------------Impelment your code here:-------------------------------------
model_relu.compile(optimizer=AdamOpt, metrics=['accuracy'], loss='categorical_crossentropy')
history_relu = model_relu.fit(BaseX_train, BaseY_train, batch_size=batch_size, epochs=epochs,verbose=2, validation_data=(BaseX_val, BaseY_val))
results_metrics = model_relu.evaluate(X_test, Y_test, batch_size=batch_size)
print("Test loss, Test accuracy:", results_metrics)
#----------------------------------------------------------------------------------------

Epoch 1/25
102/102 - 1s - loss: 1.3252 - accuracy: 0.3630 - val_loss: 1.2408 - val_accuracy: 0.5301
Epoch 2/25
102/102 - 0s - loss: 1.1845 - accuracy: 0.5748 - val_loss: 1.1275 - val_accuracy: 0.6065
Epoch 3/25
102/102 - 0s - loss: 1.0932 - accuracy: 0.6345 - val_loss: 1.0524 - val_accuracy: 0.6499
Epoch 4/25
102/102 - 0s - loss: 1.0270 - accuracy: 0.6660 - val_loss: 0.9963 - val_accuracy: 0.6881
Epoch 5/25
102/102 - 0s - loss: 0.9743 - accuracy: 0.6960 - val_loss: 0.9501 - val_accuracy: 0.7037
Epoch 6/25
102/102 - 0s - loss: 0.9317 - accuracy: 0.7102 - val_loss: 0.9134 - val_accuracy: 0.7240
Epoch 7/25
102/102 - 0s - loss: 0.8947 - accuracy: 0.7263 - val_loss: 0.8793 - val_accuracy: 0.7297
Epoch 8/25
102/102 - 0s - loss: 0.8604 - accuracy: 0.7374 - val_loss: 0.8507 - val_accuracy: 0.7373
Epoch 9/25
102/102 - 0s - loss: 0.8306 - accuracy: 0.7481 - val_loss: 0.8225 - val_accuracy: 0.7483
Epoch 10/25
102/102 - 0s - loss: 0.8032 - accuracy: 0.7576 - val_loss: 0.7987 - val_accuracy: 0.7569

---
<span style="color:red">***Task 2:***</span> *Activation functions.* 

Change the activation functions to LeakyRelu or tanh or sigmoid. Name the new model `new_a_model`. Explain how it can affect the model.*

---

In [11]:
#--------------------------Impelment your code here:-------------------------------------
new_a_model = Sequential(name="new_a_model")
new_a_model.add(Flatten(input_shape=(32,32,1)))
new_a_model.add(Dense(300,activation='sigmoid', kernel_initializer=tf.keras.initializers.he_normal()))
new_a_model.add(Dense(150))
new_a_model.add(Activation('tanh'))
new_a_model.add(Dense(4))
new_a_model.add(Activation('softmax'))
#----------------------------------------------------------------------------------------

In [12]:
new_a_model.summary()

Model: "new_a_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 300)               307500    
_________________________________________________________________
dense_4 (Dense)              (None, 150)               45150     
_________________________________________________________________
activation_2 (Activation)    (None, 150)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 604       
_________________________________________________________________
activation_3 (Activation)    (None, 4)                 0         
Total params: 353,254
Trainable params: 353,254
Non-trainable params: 0
_________________________________________________

---
<span style="color:red">***Task 3:***</span> *Number of epochs.* 

Train the new model using 25 and 40 epochs. What difference does it makes in term of performance? Remember to save the compiled model for having initialized weights for every run as we did in tutorial 12. Evaluate each trained model on the test set*

---

In [13]:
#Inputs: 
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 0
batch_size = 64
epochs = 25

#Defining the optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)



In [14]:
#--------------------------Impelment your code here:-------------------------------------
new_a_model.compile(optimizer=AdamOpt, metrics=['accuracy'], loss='categorical_crossentropy')

# saving weights:
if not("init_weights" in os.listdir()):
    os.mkdir("init_weights")
save_dir = "init_weights/"
model_name = "initial_weights"
model_path = os.path.join(save_dir, model_name)
new_a_model.save(model_path)

history_new_a = new_a_model.fit(BaseX_train, BaseY_train, batch_size=batch_size, epochs=epochs, verbose=2, validation_data=(BaseX_val, BaseY_val))
results_metrics = new_a_model.evaluate(X_test, Y_test, batch_size=batch_size)
print("Test loss, Test accuracy:", results_metrics)
#-----------------------------------------------------------------------------------------

INFO:tensorflow:Assets written to: init_weights/initial_weights/assets
Epoch 1/25
102/102 - 1s - loss: 1.3973 - accuracy: 0.3267 - val_loss: 1.3272 - val_accuracy: 0.3918
Epoch 2/25
102/102 - 0s - loss: 1.2963 - accuracy: 0.4282 - val_loss: 1.2640 - val_accuracy: 0.5191
Epoch 3/25
102/102 - 0s - loss: 1.2394 - accuracy: 0.5388 - val_loss: 1.2094 - val_accuracy: 0.5579
Epoch 4/25
102/102 - 0s - loss: 1.1906 - accuracy: 0.5544 - val_loss: 1.1628 - val_accuracy: 0.5660
Epoch 5/25
102/102 - 0s - loss: 1.1500 - accuracy: 0.5635 - val_loss: 1.1249 - val_accuracy: 0.5741
Epoch 6/25
102/102 - 0s - loss: 1.1162 - accuracy: 0.5731 - val_loss: 1.0931 - val_accuracy: 0.5845
Epoch 7/25
102/102 - 0s - loss: 1.0875 - accuracy: 0.5817 - val_loss: 1.0659 - val_accuracy: 0.6030
Epoch 8/25
102/102 - 0s - loss: 1.0633 - accuracy: 0.5965 - val_loss: 1.0440 - val_accuracy: 0.6123
Epoch 9/25
102/102 - 0s - loss: 1.0429 - accuracy: 0.5944 - val_loss: 1.0240 - val_accuracy: 0.6233
Epoch 10/25
102/102 - 0s - lo

In [15]:
#Inputs: 
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 0
batch_size = 64
epochs = 40

#Defining the optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)



In [16]:
#--------------------------Impelment your code here:-------------------------------------
from tensorflow.keras.models import load_model

new_a_model=load_model("init_weights/initial_weights")

history_new_a = new_a_model.fit(BaseX_train, BaseY_train, batch_size=batch_size, epochs=epochs, verbose=2, validation_data=(BaseX_val, BaseY_val))
results_metrics = new_a_model.evaluate(X_test, Y_test, batch_size=batch_size)
print("Test loss, Test accuracy:", results_metrics)
#-----------------------------------------------------------------------------------------

Epoch 1/40
102/102 - 1s - loss: 1.3987 - accuracy: 0.3315 - val_loss: 1.3277 - val_accuracy: 0.3918
Epoch 2/40
102/102 - 0s - loss: 1.2967 - accuracy: 0.4416 - val_loss: 1.2642 - val_accuracy: 0.5035
Epoch 3/40
102/102 - 0s - loss: 1.2399 - accuracy: 0.5185 - val_loss: 1.2105 - val_accuracy: 0.5579
Epoch 4/40
102/102 - 0s - loss: 1.1915 - accuracy: 0.5609 - val_loss: 1.1662 - val_accuracy: 0.5677
Epoch 5/40
102/102 - 0s - loss: 1.1517 - accuracy: 0.5615 - val_loss: 1.1269 - val_accuracy: 0.5741
Epoch 6/40
102/102 - 0s - loss: 1.1173 - accuracy: 0.5721 - val_loss: 1.0937 - val_accuracy: 0.5880
Epoch 7/40
102/102 - 0s - loss: 1.0884 - accuracy: 0.5845 - val_loss: 1.0668 - val_accuracy: 0.5903
Epoch 8/40
102/102 - 0s - loss: 1.0635 - accuracy: 0.5891 - val_loss: 1.0446 - val_accuracy: 0.5972
Epoch 9/40
102/102 - 0s - loss: 1.0435 - accuracy: 0.5951 - val_loss: 1.0244 - val_accuracy: 0.6244
Epoch 10/40
102/102 - 0s - loss: 1.0251 - accuracy: 0.6112 - val_loss: 1.0082 - val_accuracy: 0.6221

---
<span style="color:red">***Task 4:***</span> *Mini-batches.* 

Build the `model_relu` again and run it with a batch size of 32 instead of 64. What are the advantages of the mini-batch vs. SGD?*

---

In [17]:
keras.backend.clear_session()

In [18]:
#--------------------------Impelment your code here:-------------------------------------
model_relu = Sequential(name="model_relu")
model_relu.add(Flatten(input_shape=(32, 32, 1)))
model_relu.add(Dense(300,activation='relu', kernel_initializer=tf.keras.initializers.he_normal()))
model_relu.add(Dense(150))
model_relu.add(Activation('relu'))
model_relu.add(Dense(4))
model_relu.add(Activation('softmax'))
#----------------------------------------------------------------------------------------

In [19]:
batch_size = 32
epochs = 50

#Define your optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)


In [20]:
#--------------------------Impelment your code here:-------------------------------------
model_relu.compile(optimizer=AdamOpt,metrics=['accuracy'], loss='categorical_crossentropy')
history_relu = model_relu.fit(BaseX_train, BaseY_train, batch_size=batch_size, epochs=epochs, verbose=2, validation_data=(BaseX_val, BaseY_val))
y_pred_test = model_relu.predict(X_test)
results_metrics = model_relu.evaluate(X_test, Y_test, batch_size=batch_size)
print("Test loss, Test accuracy:", results_metrics)
#----------------------------------------------------------------------------------------

Epoch 1/50
203/203 - 1s - loss: 1.2279 - accuracy: 0.5073 - val_loss: 1.1124 - val_accuracy: 0.6047
Epoch 2/50
203/203 - 1s - loss: 1.0546 - accuracy: 0.6452 - val_loss: 1.0010 - val_accuracy: 0.6898
Epoch 3/50
203/203 - 1s - loss: 0.9674 - accuracy: 0.6954 - val_loss: 0.9309 - val_accuracy: 0.7078
Epoch 4/50
203/203 - 1s - loss: 0.9039 - accuracy: 0.7156 - val_loss: 0.8779 - val_accuracy: 0.7269
Epoch 5/50
203/203 - 1s - loss: 0.8524 - accuracy: 0.7349 - val_loss: 0.8340 - val_accuracy: 0.7407
Epoch 6/50
203/203 - 1s - loss: 0.8093 - accuracy: 0.7504 - val_loss: 0.7973 - val_accuracy: 0.7506
Epoch 7/50
203/203 - 1s - loss: 0.7739 - accuracy: 0.7615 - val_loss: 0.7654 - val_accuracy: 0.7604
Epoch 8/50
203/203 - 1s - loss: 0.7418 - accuracy: 0.7734 - val_loss: 0.7391 - val_accuracy: 0.7691
Epoch 9/50
203/203 - 1s - loss: 0.7126 - accuracy: 0.7804 - val_loss: 0.7100 - val_accuracy: 0.7801
Epoch 10/50
203/203 - 1s - loss: 0.6857 - accuracy: 0.7901 - val_loss: 0.6836 - val_accuracy: 0.7818

---
<span style="color:red">***Task 4:***</span> *Batch normalization.* 

Build the `new_a_model` again and add batch normalization layers. How does it impact your results?*

---

In [21]:
keras.backend.clear_session()

In [22]:
#--------------------------Impelment your code here:-------------------------------------
new_a_model = Sequential(name="new_a_model")
new_a_model.add(Flatten(input_shape=(32,32,1)))
new_a_model.add(Dense(300,activation='sigmoid', kernel_initializer=tf.keras.initializers.he_normal()))
new_a_model.add(BatchNormalization())
new_a_model.add(Dense(150))
new_a_model.add(BatchNormalization())
new_a_model.add(Activation('tanh'))
new_a_model.add(Dense(4))
new_a_model.add(BatchNormalization())
new_a_model.add(Activation('softmax'))
#---------------------------------------------------------------------------------------

In [23]:
batch_size = 32
epochs = 50

#Define your optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)
#Compile the network: 


In [24]:
#Preforming the training by using fit 
#--------------------------Impelment your code here:-------------------------------------
new_a_model.compile(optimizer=AdamOpt,metrics=['accuracy'], loss='categorical_crossentropy')
history_new_a = new_a_model.fit(BaseX_train, BaseY_train, batch_size=batch_size, epochs=epochs, verbose=2, validation_data=(BaseX_val, BaseY_val))
y_pred_test = new_a_model.predict(X_test)
results_metrics = new_a_model.evaluate(X_test, Y_test, batch_size=batch_size)
print("Test loss, Test accuracy:", results_metrics)
#----------------------------------------------------------------------------------------

Epoch 1/50
203/203 - 2s - loss: 1.1146 - accuracy: 0.5431 - val_loss: 1.2678 - val_accuracy: 0.3171
Epoch 2/50
203/203 - 1s - loss: 0.8384 - accuracy: 0.7047 - val_loss: 0.9787 - val_accuracy: 0.6713
Epoch 3/50
203/203 - 1s - loss: 0.7532 - accuracy: 0.7569 - val_loss: 0.7656 - val_accuracy: 0.7691
Epoch 4/50
203/203 - 1s - loss: 0.7073 - accuracy: 0.7830 - val_loss: 0.6745 - val_accuracy: 0.8194
Epoch 5/50
203/203 - 1s - loss: 0.6782 - accuracy: 0.8001 - val_loss: 0.6482 - val_accuracy: 0.8171
Epoch 6/50
203/203 - 1s - loss: 0.6440 - accuracy: 0.8202 - val_loss: 0.6368 - val_accuracy: 0.8322
Epoch 7/50
203/203 - 1s - loss: 0.6292 - accuracy: 0.8261 - val_loss: 0.6358 - val_accuracy: 0.8385
Epoch 8/50
203/203 - 1s - loss: 0.6110 - accuracy: 0.8411 - val_loss: 0.5953 - val_accuracy: 0.8547
Epoch 9/50
203/203 - 1s - loss: 0.5981 - accuracy: 0.8471 - val_loss: 0.5850 - val_accuracy: 0.8588
Epoch 10/50
203/203 - 1s - loss: 0.5865 - accuracy: 0.8545 - val_loss: 0.5821 - val_accuracy: 0.8594

### PART 2: Convolutional Neural Network (CNN)
------------------------------------------------------------------------------------

---
<span style="color:red">***Task 1:***</span> *2D CNN.* 

Have a look at the model below and answer the following:
* How many layers does it have?
* How many filter in each layer?
* Would the number of parmaters be similar to a fully connected NN?
* Is this specific NN performing regularization?

---

In [25]:
def get_net(input_shape,drop,dropRate,reg):
    #Defining the network architecture:
    model = Sequential()
    model.add(Permute((1,2,3),input_shape = input_shape))
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_1',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_2',kernel_regularizer=regularizers.l2(reg)))
    if drop:    
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_3',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_4',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_5',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    #Fully connected network tail:      
    model.add(Dense(512, activation='elu',name='FCN_1')) 
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(Dense(128, activation='elu',name='FCN_2'))
    model.add(Dense(4, activation= 'softmax',name='FCN_3'))
    model.summary()
    return model

In [26]:
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 1e-03
batch_size = 64
epochs = 25
drop = True
dropRate = 0.3
reg = 1e-2
NNet = get_net(input_shape,drop,dropRate,reg)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute (Permute)            (None, 32, 32, 1)         0         
_________________________________________________________________
Conv2D_1 (Conv2D)            (None, 32, 32, 64)        640       
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 32, 32, 64)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 128)       0

In [27]:
NNet=get_net(input_shape,drop,dropRate,reg)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 32, 32, 1)         0         
_________________________________________________________________
Conv2D_1 (Conv2D)            (None, 32, 32, 64)        640       
_________________________________________________________________
dropout_6 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 32, 32, 64)        128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
dropout_7 (Dropout)          (None, 16, 16, 128)      

In [28]:
from tensorflow.keras.optimizers import *
import os
from tensorflow.keras.callbacks import *

#Defining the optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)

#Compile the network: 
NNet.compile(optimizer=AdamOpt, metrics=['acc'], loss='categorical_crossentropy')

#Saving checkpoints during training:
Checkpath = os.getcwd()
Checkp = ModelCheckpoint(Checkpath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, save_freq=1)

In [29]:
#Preforming the training by using fit 
# IMPORTANT NOTE: This will take a few minutes!
h = NNet.fit(x=BaseX_train, y=BaseY_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0, validation_data = (BaseX_val, BaseY_val), shuffle=True)
#NNet.save(model_fn)

Epoch 1/25
102/102 [==============================] - 18s 162ms/step - loss: 8.5007 - acc: 0.3504 - val_loss: 7.9215 - val_acc: 0.2500
Epoch 2/25
102/102 [==============================] - 17s 163ms/step - loss: 7.7052 - acc: 0.5118 - val_loss: 8.0498 - val_acc: 0.2512
Epoch 3/25
102/102 [==============================] - 17s 165ms/step - loss: 7.5296 - acc: 0.5614 - val_loss: 8.0904 - val_acc: 0.2569
Epoch 4/25
102/102 [==============================] - 17s 164ms/step - loss: 7.4107 - acc: 0.5819 - val_loss: 8.1286 - val_acc: 0.2714
Epoch 5/25
102/102 [==============================] - 16s 159ms/step - loss: 7.2934 - acc: 0.6304 - val_loss: 8.0654 - val_acc: 0.3056
Epoch 6/25
102/102 [==============================] - 16s 159ms/step - loss: 7.1918 - acc: 0.6543 - val_loss: 7.8972 - val_acc: 0.3345
Epoch 7/25
102/102 [==============================] - 16s 157ms/step - loss: 7.1311 - acc: 0.6744 - val_loss: 7.8012 - val_acc: 0.3640
Epoch 8/25
102/102 [==============================] - 1

In [30]:
# NNet.load_weights('Weights_1.h5')

In [31]:
results = NNet.evaluate(X_test,Y_test)
print('test loss, test acc:', results)

6/6 [==============================] - 0s 18ms/step - loss: 7.9150 - acc: 0.3371
test loss, test acc: [7.9150285720825195, 0.33714285492897034]


---
<span style="color:red">***Task 2:***</span> *Number of filters* 

Rebuild the function `get_net` to have as an input argument a list of number of filters in each layers, i.e. for the CNN defined above the input should have been `[64, 128, 128, 256, 256]`. Now train the model with the number of filters reduced by half. What were the results.

---

In [32]:
#--------------------------Impelment your code here:-------------------------------------
def get_net(input_shape, drop, dropRate, reg, filters):
    #Defining the network architecture:
    model = Sequential()
    model.add(Permute((1, 2, 3), input_shape=input_shape))
    model.add(Conv2D(filters=filters[0], kernel_size=(3, 3), padding='same', activation='relu', name='Conv2D_1', kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=filters[1], kernel_size=(3, 3), padding='same', activation='relu', name='Conv2D_2', kernel_regularizer=regularizers.l2(reg)))
    if drop:    
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=filters[2], kernel_size=(3, 3), padding='same', activation='relu',name='Conv2D_3',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=filters[3], kernel_size=(3, 3), padding='same', activation='relu',name='Conv2D_4', kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=filters[4], kernel_size=(3, 3), padding='same', activation='relu',name='Conv2D_5', kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    #Fully connected network tail:      
    model.add(Dense(512, activation='elu',name='FCN_1')) 
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(Dense(128, activation='elu',name='FCN_2'))
    model.add(Dense(4, activation= 'softmax',name='FCN_3'))
    model.summary()
    return model

filters = [32, 64, 64, 128, 128]
NNet_new=get_net(input_shape,drop,dropRate,reg,filters)
NNet_new.compile(optimizer=AdamOpt, metrics=['acc'], loss='categorical_crossentropy')
h_new = NNet_new.fit(x=BaseX_train, y=BaseY_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0, validation_data=(BaseX_val, BaseY_val), shuffle=True)
results_metrics = NNet_new.evaluate(X_test,Y_test)
print('Test loss, Test acc:', results_metrics)
#----------------------------------------------------------------------------------------

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_2 (Permute)          (None, 32, 32, 1)         0         
_________________________________________________________________
Conv2D_1 (Conv2D)            (None, 32, 32, 32)        320       
_________________________________________________________________
dropout_12 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_13 (Dropout)         (None, 16, 16, 64)       

That's all folks! See you :)